# Исследование надежности заемщиков.

Кредитный отдел банка заинтересован в том, чтобы иметь возможность оценить надежность своего потенциального заемщика.

Изучим статистику о платёжеспособности клиентов, предоставленную банком, и попытаемся определить факторы, способные влиять на кредитоспособность.

**Цель исследования** — <u>проверить 4 гипотезы:</u> <a id='hypothesis'></a>
1. На способность потенциального заемщика погасить кредит в срок влияет его семейный статус.
2. Количество детей клиента влияет на факт погашения кредита в срок.
3. Между уровнем дохода и возвратом кредита в срок сщуествует зависимость.
4. Разные цели кредита по-разному влияют на его возврат в срок.

**Ход исследования**
Данные о платежеспособности клиентов получим из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор и предобработка данных, в случае, если в них будут обнаружены артефакты.

Начнем с проверки данных на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки постараемся исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез .

## Обзор данных <a id='intro'>

Составим первое представление о данных, предоставленных банком.

Для работы с датафреймом нам потребуется библиотека pandas, импортируем ее:

In [44]:
import pandas as pd 

In [4]:
df = pd.read_csv('/datasets/data.csv') # cчитаем файл data.csv из папки /datasets и сохраним его в переменной df
df.head(10) # выведем на экран первые 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


На первый взгляд, критичных отклонений в данных нет, за исключением аномалии в столбце `days_employed` - отрицательных количеств дней трудового стажа. Смущает, что дни выражены в вещественных числах. Также бросаются в глаза дубликаты в столбце `education`, записанные в разных регистрах.

Рассмотрим общую информацию о данных в таблице df:

In [5]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Используются 3 типа данных: `float64`, `int64`, `object`.

Согласно документации к данным:
* **`children`** — количество детей в семье 
* **`days_employed`** — общий трудовой стаж в днях 
* **`dob_years`** — возраст клиента в годах 
* **`education`**— уровень образования клиента
* **`education_id`** — идентификатор уровня образования
* **`family_status`** — семейное положение
* **`family_status_id`** — идентификатор семейного положения
* **`gender`** — пол клиента
* **`income_type`** — тип занятости
* **`debt`** — имел ли задолженность по возврату кредитов
* **`total_income`** — ежемесячный доход
* **`purpose`** — цель получения кредита

В столбцах `days_employed` и `total_income` есть пропущенные значения. 

**Выводы**

В каждой строке таблицы лежат данные о кредитозаемщике банка. Бóльшая часть колонок описывает личную информацию о клиенте, такую, как возраст, семейное положение, ежемесячный доход и т.д. Столбцы `debt` и `purpose` дают представление о наличии задолженности по возврату кредита и цели его получения соответственно.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Однако встречаются пропуски в данных, чтобы двигаться дальше, устраним эту проблему.

## Предобработка данных

### Заполнение пропусков <a id='abc'></a>

Посчитаем, сколько в таблице пропущенных значений. Для этого воспользуемся двумя методами pandas (isna() - чтобы определить пустые ячейки и sum() - для подсчета их количества в каждом столбце) :

In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы убедились в том, что в столбцах `days_employed` и `total_income` есть пропущенные значения.

Рассчитаем долю пропущенных значений для стобца `days_employed`. Так как количество пропусков в обоих столбцах одинаково, пропуски в колонке `total_income` будут составлять такую же долю.

In [7]:
# Чтобы посчитать долю пропусков (isna_share) - разделим кол-во пропусков на длинну столбца:
isna_share=len(df[df['days_employed'].isna()])/len(df['days_employed'])
# Округлим значение до десятых:
round(isna_share, 1)

0.1

Таким образом, в колнках `days_employed` и `total_income` есть пропущенные значения, доля которых составляет 10% в каждом из столбцов.
Рассмотрим первые 10 строк с пропущенными значениями в с толбце `days_employed` и попытаемся понять причину отсутствия этих значений в таблице.

In [8]:
# Выведем первые 10 строк с пропущенными значениями с  помощью функций isna() и head()
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Кажется, что данные о трудовом стаже и месячном доходе отсутствуют у одних и тех же людей. Проверим это, посчитав среднее арифметическое значений `total_income` в тех series, где у `days_employed` отсутствуют значения:

In [9]:
df[df['days_employed'].isna()]['total_income'].mean()

nan

Действительно, во всех строках, где значения `days_employed` отсутствуют, также отсутствуют и значения `total_income`. Учитывая, что в обоих столбцах одинаковое количество пропусков, делаем вывод, что данные о трудовом стаже и месячном доходе пропущенны у одних и тех же людей. Однако другие взаимосвязи в данной выборке мы не наблюдаем. Сложно предположить, что заемщики сами не посчитали нужным предоставлять данную информацию, так как документы, подтверждающие финансовое состояние и трудовую занятость заемщика требуются банками в обязательном порядке. Следовательно, вероятнее всего, пропуски возникли по технологическим причинам.

В то время, как значения трудового стажа вовсе не повлияют на наше исследование, пропуски в данных о ежемесячных доходах клиентов важны для проверки [3ей гипотизы](#hypothesis).  

Возможности установить точную причину пропусков и восстановить данные у нас нет. Отсутствующие значения в обоих колонках логично заполнить медианными значениями, так как:
1. удаление более 1% пропусков от всех значений датафрейма (в нашем случае 10%) может повлиять на точность исследования, поэтому, чтобы не терять данные, пропуски лучше заполнить; 
2. переменные, содержащиеся в обоих колонках, являются количественными;
3. значения уровня месячного дохода и трудового стажа отдельных клиентов могут сильно выделяться среди большинства, из чего следует, что медиана, в отличае от среднего арифметического, позволит нивелировать влияние таких отклонений. 

Пропуски в трудовом стаже и в ежемесячном доходе логичнее заполнять не медианой от всех значений по столбцу соттветственно, а по гуппам типа занятости. Для этого создадим функцию `fillna_median_by_group`, которая будет принимать:
- колонку с пропусками - `column_1`
- колонку, по которой осуществляется группировка - `column_2`.
Что сделает наша функция:
- выдаст количество пропусков в `column_1`;
- с помощью `unique()` разобъет `column_2`на группы;
- с `loc` сделает срезы по выделенным группам, где пропущенны значения `column_1`;
- найдет медиану по группе и заполнит ей пропуски в этой группе;
- выдаст оставшееся количество пропусков.

In [10]:
def fillna_median_by_group(column_1, column_2):

    print('Пропуски до:', df[column_1].isna().sum())

    for type in df[column_2].unique():
        median = df.loc[df[column_2] == type, column_1].median()
        print(type, median)
        df.loc[(df[column_1].isna()) & (df[column_2] == type), column_1] = median
    print('Пропуски после:', df[column_1].isna().sum())

Применим функцию `fillna_median_by_group` к `days_employed` и к `total_income` соответственно:

In [11]:
print(fillna_median_by_group('days_employed', 'income_type'))
print()
fillna_median_by_group('total_income', 'income_type')  

Пропуски до: 2174
сотрудник -1574.2028211070851
пенсионер 365213.30626573117
компаньон -1547.3822226779334
госслужащий -2689.3683533043886
безработный 366413.65274420456
предприниматель -520.8480834953765
студент -578.7515535382181
в декрете -3296.7599620220594
Пропуски после: 0
None

Пропуски до: 2174
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Пропуски после: 0


Благодаря функции, пропусков в данных колонках не осталось. Финально убедимся, что во всем датафрейме больше нет пропущенных значений:

In [12]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления. <a id='anomalies'></a>

В ходе [Обзора данных](#intro) мы обнаружили артефакт в общем трудовом стаже заемщика в днях - отрицательное количество дней трудового стажа. Вероятнее всего, знак "-" появился случайно. Избавимся от знака минус - с помощью `abs()`, вычислив модуль и выведем первые 5 значений столбца `days_employed`:


In [13]:
df['days_employed'] = abs(df['days_employed'])
df['days_employed'].head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

Бросается в глаза значение под индексом 4. Посчитаем, сколько примерно лет проработал этот человек:

In [14]:
df['days_employed'][4]//365

932.0

932 года - кажется, чересчур. Интересно, единственный ли "долгожитель" в нашем датафрейме? Гипотетически представим человека, который проработал с 18 до 65 лет, примем его стаж за условный максимум.

In [15]:
# посчитаем условный максимальный стаж работы в днях:
print('Условный максимальный стаж работы: ',(65-18)*365, 'дней') 


Условный максимальный стаж работы:  17155 дней


In [16]:
# посчитаем количество людей, чей стаж превышает условный максимум:
print('Приблизительная доля людей, чей стаж превышает условный максиму:', len(df[df['days_employed'] > 17155])/len(df)*100,'%')
# отсортируем датафрейм по трудовому стажу в порядке убывания и выведем 10 первых значений
df.sort_values(by='days_employed',ascending=False).head(10) 

Приблизительная доля людей, чей стаж превышает условный максиму: 17.932636469221837 %


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


Мы видим, что людей, чей стаж работы указан неверно - значительное колличество, а именно 18%.

Предположим, что у "долгожителей" трудовой стаж указан в месяцах. Посчитаем, сколько примерно лет проработал такой заемщик:

In [17]:
df[df['days_employed'] > 17155]['days_employed'].median()//12

30434.0

Более 30 тысяч лет - невозможно, значит здесь временная единица - точно не месяц. Проверим также часы и минуты по порядку:

In [18]:
# на 8 часов - получим кол-во рабочих дней и 365 - получим кол-во лет
print(df[df['days_employed'] > 17155]['days_employed'].median()/8/365) 

# на 60 минут - получим часы, на 8 часов - получим кол-во рабочих дней и 365 - получим кол-во лет
df[df['days_employed'] > 17155]['days_employed'].median()/60/8/365

125.07305009100382


2.084550834850064

125 лет - слишком много, 2 года - тоже маловероятно.

Несмотря на то, что мы столкнулись с очевидной масштабной ошибкой в данных, удалить все строки, где есть ошибка мы не можем, так как рискуем утратить другую, полезную для исследования информацию. Мы убедились, что временная единица в стаже у "долгожителей" - не месяцы и не часы. Также маловероятно, что она указана минутах.
Чтобы разобраться с данной аномалией, предстоит обратиться к сотрудникам банка.

Проверим на наличие артефактов клювые для нашего исследования столбцы, а именно:
- `family_status` 
- `children`
- `total_income` 
- `purpose` 

Выведем все уникальные значения колонки семейного положения с помощью функции `unique()`:

In [19]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Мы убедились, что в данном столбце нет аномальных значений или дубликатов. Есть расхождение в регистрах: `'Не женат / не замужем'` написано с заглавной буквы, устсраним его с помощью метода `str.lower()`:

In [20]:
df['family_status'] = df['family_status'].str.lower() 
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Перейдем к колонке, обозначающую количество детей в семье, выведем уникальные значения в порядке возрастания:

In [21]:
# отсортируем значения и сохраним в переменную sorted_by_amount:
sorted_by_amount = df.sort_values(by='children') 
# выведем уникальные значения:
sorted_by_amount['children'].unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

Значения `-1` и `20` кажутся как минимум нереалистичными, с ними стоит разобраться. Узнаем, как часто встречаются данные значения:

In [22]:
print('Количество заемщиков с -1 ребенком:', len(df[df['children'] == -1]))
print('Количество заемщиков с 20 детьми:', len(df[df['children'] == 20]))

Количество заемщиков с -1 ребенком: 47
Количество заемщиков с 20 детьми: 76


В данном случае, вероятнее всего, причиной ошибок служат опечатки: заменим значения `-1` на `1`, а `20` на `2` с помощью функции `replace()`:

In [23]:
df['children'] = df['children'].replace(to_replace = -1, value = 1)
df['children'] = df['children'].replace(to_replace = 20, value = 2)
# Выведем уникальные значения столбца, чтобы убедиться, замена значений сработал:
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Рассмотрим столбец `total_income` на наличие аномальных значений. Выведем минимальное и максимальное значения (с помощью функций `min()` и `max()`), чтобы понимать насколько реалистечен зарплатный диапазон:

In [24]:
print('Минимальное значение ежемесчного дохода:',df['total_income'].min())
print('Максимальное значение ежемесчного дохода:',df['total_income'].max())

Минимальное значение ежемесчного дохода: 20667.26379327158
Максимальное значение ежемесчного дохода: 2265604.028722744


Значения не вызывают сомнений: доходы заемщиков варьируются от 20667 до 2265604 рублей (в документации значение валюты не указано, но помнимаем, что вероятнее всего это рубли).

Проверим последний, важный для нашего исследования столбец - `purpose`, выведем все уникальные значения:

In [25]:
df['purpose'].sort_values().unique() 

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

В данных о целях получения кредита аномалий нет.

Также рассмотрим `dob_years` на наличие аномальных значений:

In [26]:
df['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

В наш список заемщиков попали "груднички", которым 0 лет. Узнаем, их долю от всех заемщиков:

In [27]:
len(df.query('dob_years == 0'))/len(df)


0.004692218350754936

 Меньше процента — это немного, в нашем исследовании удаление такой доли строк не повлияет на результаты. 

### Изменение типов данных.

На этапе знакомства с данными мы обнаружили, что значения общего трудовой стаж в днях (`days_employed`) и ежемесячного дохода в рублях (`total_income`) выражены в вещественных числах. Такой формат не воспринимается наглядно и может затруднить дальнейшую работу с данными. С помощью метода `astype()` переведем значения в тип `int`:

In [28]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
# с помощью метода .dtypes убедимся, что тип данных изменился:
df[['days_employed','total_income']].dtypes 


days_employed    int64
total_income     int64
dtype: object

### Удаление дубликатов.   

Посчитайем явные дубликаты в таблице командой sum():

In [29]:
df.duplicated().sum()

54

Удалим явные дубликаты с помощью метода drop_duplicates(), а с помощью метода reset_index() обновим индексы:

In [30]:
df = df.drop_duplicates().reset_index(drop=True)
# ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:
df.duplicated().sum()

0

Дублирующиеся строки, вероятнее всего, появились по технологической ошибки, например, при копирование данных.

Рассмотрим столбец `education`, выведем все уникальные значения, чтобы выявить неявные дубликаты:

In [31]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Избавиться от этих дубликатов логичнее всего с помощью приведения всех значений к нижнему регистру, используем метод `str.lower()`:

In [32]:
df['education'] = df['education'].str.lower()
df['education'].unique() # выведем уникальные значения, чтобы убедиться, что дубликатов больше нет

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Расхождения в регистре - ошибка, возникшая в результате человекческого фактора: данные вносили разные люди, кто-то не придерживался общего стиля.

Финально проверим датасет на дубликаты:

In [33]:
df.duplicated().sum()

17

Все чисто.

###  Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма-словаря:
- `education` c идентификаторами уровня образования
- `family_status` c идентификаторами семейного положения

In [34]:
# добавим в новый датафрейм столбцы об образовании
education = df[['education_id','education']] 
# удалим индексы
education.set_index('education_id', inplace=True)
# удалим дубликаты
education = education.drop_duplicates()

# добавим в новый датафрейм столбцы о семейном положении
family_status = df[['family_status_id', 'family_status']] 
# удалим индексы
family_status.set_index('family_status_id', inplace=True)
# удалим дубликаты
family_status = family_status.drop_duplicates()

# выведем получившиеся словари
display(education)
family_status

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,не женат / не замужем


Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. Выведем обновленный перечень столбцов.

In [35]:
df.drop(['education','family_status'], axis=1, inplace=True) 
list(df)

['children',
 'days_employed',
 'dob_years',
 'education_id',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

### Категоризация дохода.

Напишем функцию `total_income_category`, которая возвращает категорию дохода по значению столбца `total_income`, используя правила:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [36]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income >= 30001 and total_income < 50000:
        return 'D'
    if total_income >= 50001 and total_income < 200000:
        return 'C';
    if total_income >= 200001 and total_income < 1000000:
        return 'B';
    return 'A'
# проверим работу функции на двух значениях.
print(total_income_category(25000))
total_income_category(235000)

E


'B'

Создадим отдельный столбец с доходными категориями и выведем первые 5 строк датафрейма:

In [37]:
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Категоризация целей кредита.

Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию для целей кредита.

In [38]:
df['purpose'].sort_values().unique() # выведем отсортированные уникальные значения

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Создадим функцию `purpose_category`, которая на основании цели кредита выведет соответствующую категорию из четырех:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

In [39]:
def purpose_category(purpose):
    # создадим 4 условия, на основании которых функция будет выбирать соответствующую категорию:   
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose or 'недвижим' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образова' in purpose:
        return 'получение образования'
    return ('Ошибка! Цель кредита не попадает ни под одну категорию.')
# cоздадим отдельный столбец для категорий целей, применим к нему функцию и выведем первые 5 строк датафрейма:
df['purpose_category'] = df['purpose'].apply(purpose_category)
df.head(5)  

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


**Выводы**

Предобработка обнаружила четыре проблемы в данных:

- пропущенные значения,
- аномалии,
- нарушения в типах данных,
- дубликаты — явные и неявные.

Мы исправили ошибки, чтобы упростить дальнейшую работу с таблицей. Без дубликатов, пропусков, неверных типов данных исследование станет более точным.

Значительная часть ошибок произошла по вине человекческого фактора. Сотрудниками банка, которые работают с данной таблицей и вносят новые данные, можно порекоммендовать пользоваться единым шаблоном, чтобы избежать новых ошибок.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотиз

##### Проверка первой гипотизы

***Гипотиза №1: На способность потенциального заемщика погасить кредит в срок влияет его семейный статус.***

Для того, чтобы подтвердить, или опровергнуть данную гипотизу, нам необходимо понять, какой процент заемщиков имеет задолженность в каждой категории семейного положения. Для этого создадим сводную таблицу `pivot_family_status` c подсчетами для каждой категории.

In [40]:
'''создадим новый датафрейн, в который добавим 2 столбца: 
идентификатор семейного положения и имел/не имел задолженность'''
family_status_debt = df[['family_status_id', 'debt']]

# к этой таблице с помощбю метода merge() присоединим словарь family_status:
family_status_debt = family_status_debt.merge(family_status, on='family_status_id', how='left')

# создадим сводную таблицу:
pivot_family_status = family_status_debt.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')

# добавим столбец total со значением общего количества заемщиков для конкретной категории:
pivot_family_status['total'] = pivot_family_status[0] + pivot_family_status[1] 

''' добавим столбец percentage of debtors, отражающий процент людей в конкретной категории, 
которые имеют задолженности'''
pivot_family_status['percentage of debtors'] = (pivot_family_status[1] / pivot_family_status['total']*100)

# отсортируем столбец percentage of debtors по убыванию
pivot_family_status.sort_values(by = 'percentage of debtors', ascending = False)


debt,0,1,total,percentage of debtors
family_status,,,,
не женат / не замужем,2536,274,2810,9.750890
гражданский брак,3775,388,4163,9.320202
женат / замужем,11413,931,12344,7.542126
в разводе,1110,85,1195,7.112971
вдовец / вдова,896,63,959,6.569343


##### Вывод 1:

Процент должников по всем 5 категориям семейного положения приблизительно варьируется от 7 до 10 процентов. 

Наиболее достоверным расчетом можно считать процент клиентов, имевших задолженность по кредиту, в категории `женат / замужем`, так как это самая многочисленная категория (12344 человека). Можно утверждать, что человек, состоящий в браке с вероятностью 8% будет иметь задолженность.

Выборки не состоящих в браке людей и проживающих в гражданском браке значительно меньше, чем женатых/замужних - 2810 и 4163 человека соответственно. Однако именно среди этих двух выборок клиенты с бóльшей вероятностью имеют задолженности примерно 10% и 9%, согласно нашим рассчетам. Поэтому, мы можем только предположить, 'свободные' люди, или люди, состоящие в неузаконенных отношениях, являются менее надежными клиентами для банка.

Согласно сводной таблице, клиенты в разводе и овдовевшие люди - наиболее 'пунктальные' кредиторы, но не стоит забывать, что это - самые маленькие выборки (`в разводе` - 1195 человек, `вдовец / вдова` - 959 человек), поэтому данное утверждение имеет меньший вес.

**Таким образом, яркой корреляции между семейным положением и вероятностью человека вернуть кредит в срок нет.**  


##### Проверка второй гипотизы

***Гипотиза №2: Количество детей клиента влияет на факт погашения кредита в срок.***


Для того, чтобы подтвердить, или опровергнуть данную гипотизу, нам необходимо понять, какой процент заемщиков имеет задолженность в каждой категории количества детей в семье. Для этого создадим сводную таблицу `pivot_children` c подсчетами для каждой категории.

In [41]:
# создадим сводную таблицу:
pivot_children = df.pivot_table(index='children', columns='debt', values='gender', aggfunc='count')

# заменим Nan в сводной
pivot_children = pivot_children.fillna(0) 

# добавим столбец total со значением общего количества заемщиков для конкретной категории:
pivot_children['total'] = pivot_children[0] + pivot_children[1] 

''' добавим столбец percentage of debtors, отражающий процент людей в конкретной категории, 
которые имеют задолженности'''
pivot_children['percentage of debtors'] = (pivot_children[1] / pivot_children['total'] * 100)

# изменим тип данных в остальных столбцах:
pivot_children[[0,1,'total']] = pivot_children[[0,1,'total']].astype('int')

# отсортируем столбец percentage of debtors по убыванию:
pivot_children.sort_values(by = 'percentage of debtors', ascending = False)

debt,0,1,total,percentage of debtors
children,,,,
4,37,4,41,9.756098
2,1926,202,2128,9.492481
1,4411,445,4856,9.163921
3,303,27,330,8.181818
0,13044,1063,14107,7.535266
5,9,0,9,0.000000


##### Вывод 2:

Процент должников по всем 6 категориям количества детей в семье приблизительно варьируется от 7,5 до 10 процентов.

Самая большая выборка - заемщики, не имеющие детей - 13044 человека. Соответственно, говоря о людях из этой категории, можно с бóльшей достоверностью утверждать, что человек без детей будет иметь задолженность по кредиту с вероятностью 8%.

Самая маленькая выборка - клиенты, у которых по 5 детей (9 человек), среди которых нет должников. Действительно, такие многодетные семьи редкость. Учитывая размер выборки, сделать однозначное заключение о том, что у данных заемщиков самая маленькая вероятность просрочить кредит нельзя.

Остальные категории по своей численности варьируются от 41 до 4856 человек, при этом процент должников в них практически одинаковый (от 8 до 10%). Самая маленькая выборка (клиенты с 4 детьми) в стро двадцать раз меньше, чем кол-во клиентов с 1 ребенком. Учитывая такой разброс, сложно сделать достоверные выводы.

**Таким образом, яркой корреляции между количеством детей в семье заемщика и его вероятностью вернуть кредит в срок нет. Единственное, что можно заметить, что у клиента с 4 детьми с бóльшей вероятностью будет задолженность по кредиту, чем у клиента с 5 детьми.**

##### Проверка третей гипотизы

***Гипотиза №3: Между уровнем дохода и возвратом кредита в срок сщуествует зависимость.***

Для того, чтобы подтвердить, или опровергнуть данную гипотизу, нам необходимо понять, какой процент заемщиков имеет задолженность в каждой категории месячного дохода. Для этого создадим сводную таблицу `pivot_total_income` c подсчетами для каждой категории.

In [42]:
# создадим сводную таблицу:
pivot_total_income  = df.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')

# добавим столбец total со значением общего количества заемщиков для конкретной категории:
pivot_total_income['total'] = pivot_total_income[0] + pivot_total_income[1] 

''' добавим столбец percentage of debtors, отражающий процент людей в конкретной категории, 
которые имеют задолженности:'''
pivot_total_income['percentage of debtors'] = (pivot_total_income[1] / pivot_total_income['total'] * 100)

# выведем сводную таблицу
pivot_total_income

debt,0,1,total,percentage of debtors
total_income_category,,,,
A,23,2,25,8.000000
B,4686,356,5042,7.060690
C,14672,1360,16032,8.483034
D,329,21,350,6.000000
E,20,2,22,9.090909


*Классификации доходов:*
- 1000001 и выше — 'A';
- 200001–1000000 — 'B';
- 50001–200000 — 'C';
- 30001–50000 — 'D';
- 0–30000 — 'E'.

##### Вывод 3:

Большая часть клиентов (16033) имеет доход категории С - 50001–200000 условной валюты. 8% из них имели задолженности. 
Эта выборка значительно превышает остальные категории, и данный подсчет можно назвать самым достоверным.
Наибольшая вероятность (9%), что заемщик просрочит кредит среди людей, чей доход не превышает 30000. Несмотря на то, что это самая маленькая выборка (22 человека), данный подсчет кажется логичным: людям с самым маленьким доходом сложнее всех вернуть кредит в срок.
Наименьшая доля должников (6%) среди людей с категорией дохода D (от 30000 до 50000).
У первых двух категорий, заемщиков, чей доход превышает 200000, доля должников практически одинаковая: 7-8%. При этом численность выборки А в 200 раз больше, чем выборки В.

**Таким образом, яркой корреляции между уровнем дохода заемщика и его вероятностью вернуть кредит в срок нет. Тем не менее, можно предположить, что люди с самым низким доходом (до 30000) с наибольшей вероятностью просрочат выплату.**

##### Проверка четвертой гипотизы

***Гипотиза №4: Разные цели кредита по-разному влияют на его возврат в срок.***

Для того, чтобы подтвердить, или опровергнуть данную гипотизу, нам необходимо понять, какой процент заемщиков имеет задолженность в каждой категории цели получения кредита. Для этого создадим сводную таблицу pivot_purpose c подсчетами для каждой категории.

In [43]:
# создадим сводную таблицу:
pivot_purpose = df.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')

# добавим столбец total со значением общего количества заемщиков для конкретной категории:
pivot_purpose['total'] = pivot_purpose[0] + pivot_purpose[1] 

''' добавим столбец percentage of debtors, отражающий процент людей в конкретной категории, 
которые имеют задолженности:'''
pivot_purpose['percentage of debtors'] = (pivot_purpose[1] / pivot_purpose['total']) * 100

# выведем отсортированную сводную таблицу
pivot_purpose.sort_values(by = 'percentage of debtors', ascending = False)

debt,0,1,total,percentage of debtors
purpose_category,,,,
операции с автомобилем,3905,403,4308,9.354689
получение образования,3644,370,4014,9.217738
проведение свадьбы,2149,186,2335,7.965739
операции с недвижимостью,10032,782,10814,7.231367


##### Вывод 4:

Мы можем наблюдать, что наибольшее количество кредиторов, не вернувших кредит в срок, среди людей, чья цель кредита связана с операцией с автомобилем и с получением образования - 9% от этих заемщиков. 
В то же время, наименьший процент задолженников среди тех, кто брал кредит для проведения операции с недвижимостью. 
**Выборки в данном исследовании соизмерими, поэтому можно с уверенностью утверждать, что наиболее 'безопасными' заемщиками для банка являются те, кто брали кредит для операций с недвижимостью. Наименее привлекательные клиенты для банка те, кому нужен кредит для операции с автомобилем, или для получения образования. Гипотиза подтверждается.**

## Общий вывод:

Благодаря предобработке, мы обнаружили **четыре основные проблемы в данных:**

- пропущенные значения,
- аномалии,
- нарушения в типах данных,
- дубликаты — явные и неявные.

Значительная часть ошибок произошла по вине человекческого фактора. Сотрудниками банка, которые работают с предоставленной нам таблицей и вносят новые данные, можно порекоммендовать пользоваться единым шаблоном, чтобы избежать новых ошибок.

**Были исправлены все проблемы, за исключением:**
- должников, чей возраст = 0 лет (таких кредиторов меньше 1%);
- трудового стажа "долгожителей" (более 18%).

Для устранения этих ошибок необходимо обратиться за информацией к сотрудникам банка.


**Очистив данные, мы проверили четыре гипотезы, 3 из которых не подтвердились на основании предоставленных данных:**

1. На способность потенциального заемщика погасить кредит в срок влияет его семейный статус.
2. Количество детей клиента влияет на факт погашения кредита в срок.
3. Между уровнем дохода и возвратом кредита в срок сщуествует зависимость.

Причиной этому может быть тот факт, что гипотизы неверны, либо корреляция совсем незначительна, для того, чтобы ее отследить, нужно больше данных и более соизмеримые выборки по категориям.

**Последняя гипотеза полностью подтвердилась.**

4. Разные цели кредита, хоть и незначительно, но по-разному влияют на его возврат в срок:

наиболее 'безопасными' заемщиками для банка являются те, кто брали кредит для операций с недвижимостью. Наименее привлекательные клиенты для банка те, кому нужен кредит для операции с автомобилем, или для получения образования.